In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install scipy


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Caminhos para as pastas de dados
train_data_dir = './data'
batch_size = 32 # ***
image_size = (150, 150)
num_epochs = 50
num_classes = 2

In [7]:
# Crie geradores de dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, # Normalização das imagens
    validation_split=0.25,  # Define a divisão de validação
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)    

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 16 images belonging to 2 classes.


In [9]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 4 images belonging to 2 classes.


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Camada de saída com 1 unidade para classificação binária

In [11]:
# Compile o modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
# Treine o modelo
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator)

# Salve o modelo treinados
model.save('./models/cfModeloTales.h5')

print('Modelo salvo com sucesso!')

Epoch 1/50
1/1 [==============================] - 0s 495ms/step - loss: 0.0571 - accuracy: 0.9375 - val_loss: 0.8636 - val_accuracy: 0.7500
Epoch 2/50
1/1 [==============================] - 0s 439ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 1.0507 - val_accuracy: 0.7500
Epoch 3/50
1/1 [==============================] - 0s 432ms/step - loss: 0.0487 - accuracy: 1.0000 - val_loss: 1.2317 - val_accuracy: 0.7500
Epoch 4/50
1/1 [==============================] - 0s 429ms/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 1.0041 - val_accuracy: 0.7500
Epoch 5/50
1/1 [==============================] - 0s 448ms/step - loss: 0.0280 - accuracy: 1.0000 - val_loss: 1.3057 - val_accuracy: 0.7500
Epoch 6/50
1/1 [==============================] - 0s 411ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.7562 - val_accuracy: 0.7500
Epoch 7/50
1/1 [==============================] - 0s 442ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 1.1251 - val_accuracy: 0.7500
Epoch 8/50
1/1 [====

c:\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
